In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tools
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from datetime import datetime
from typing import Annotated
from typing_extensions import TypedDict

from dotenv import load_dotenv
_ = load_dotenv()


In [ ]:
llm = ChatOpenAI(model="gpt-4o")


In [ ]:
tavily_search = TavilySearchResults(max_results=2)

@tool
def get_current_date():
    """Returns the current date and time. Use this tool first for any 'time-based' questions."""
    return f"The current date is: {datetime.now().strftime('%d %B %Y')}"


    

In [ ]:
tools = [tavily_search, get_current_date]

In [ ]:
llm_with_tools= llm.bind_tools(tools)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
## Create chatbot here....

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
## Have to have a graph first to use the chatbot node
graph_builder = StateGraph(State)

In [ ]:
graph_builder.add_node("chatbot", chatbot)


In [ ]:
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)

In [ ]:
### NOTE FOR MYSELF: This is a conditional edge Im making to do the actual decision whether to use the tools/functions or
## not. This in particular is a prebuilt conditional edge...
graph_builder.add_conditional_edges("chatbot", tools_condition)

In [ ]:
## Now if the tools are not used,
## then Im just going to return to the chatbot and process the tool output...
graph_builder.add_edge("tools","chatbot")

In [ ]:
## NOTE: You have to set the entry point to the graph cycle...

graph_builder.set_entry_point("chatbot")

In [ ]:
## compile graph here...

graph = graph_builder.compile()


In [ ]:
from IPython.display import display, Image

# Visualize the graph...

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass ## visualization needs more stuff...



In [ ]:
## run graph here...

from IPython.display import Markdown, display

def render_markedown(md_string):
    display(Markdown(md_string))

def process_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message,tuple):
            print(message)
        else:
            message.pretty_print()
    return message

def process_query(query, config=None):
    inputs = {"messages": [("user", query)]}
    message = process_stream(graph.stream(inputs, config, stream_mode="values"))
    render_markdown(f"## Answer:\n{message.content}")







